In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cantera as ct
import time
import rk_solver_cpp
from scipy.integrate import solve_ivp
from tqdm import tqdm

In [2]:
def setup_gas(T, pp):
    gas = ct.Solution('mechanism_files/ch4_53species.yaml')
    gas.TP = T, ct.one_atm*pp
    gas.set_equivalence_ratio(1.0, 'CH4', 'O2:1.0, N2:3.76')
    return gas, pp 

def combustion_ode(gas, pp):
    def f(t, Y):
        T = Y[0]
        YY = Y[1:]
        gas.TPY = T, ct.one_atm*pp, YY
        species_rates = gas.net_production_rates*gas.molecular_weights/gas.density
        species_h = gas.partial_molar_enthalpies/gas.molecular_weights
        temp_rate = -np.sum(species_rates*species_h/gas.cp_mass)
        return np.concatenate((np.array([temp_rate]), species_rates), axis=0)
    return f

def detect_ignition(Temperatures, tolerance=1e-3):
    return np.any(np.diff(Temperatures) > tolerance)




In [3]:
T_initial = 1400
pp_initial = 40
gas, pp = setup_gas(T_initial, pp_initial)
t_end = 2e-4
dt = 1e-6

npoints = int(t_end/dt)

t_span = np.linspace(0, t_end, npoints + 1)

f = combustion_ode(gas, pp)∫∫∫
y0 = np.hstack([[gas.T], gas.Y])


In [4]:
def solve_with_cpp(f, t_span, y0, rtol=1e-6, atol=1e-8):
    rk23 = rk_solver_cpp.RK23(f, t_span[0], y0, t_span[-1], rtol=rtol, atol=atol)
    print(f"Solving with C++ RK23")
    start_time = time.time()
    try:
        
        result_cpp = rk_solver_cpp.solve_ivp(rk23, t_span)
        time_cpp = time.time() - start_time
        cpp_success = result_cpp['success']
        if cpp_success:
            ys_cpp = result_cpp['y']
        else:
            print(f"C++ RK23 failed: {result_cpp['message']}")
        return ys_cpp, time_cpp, cpp_success
    except Exception as e:
        print(f"C++ RK23 failed: {str(e)}")
        time_cpp = None
        cpp_success = False
        return None, None, cpp_success

In [10]:
for t in t_span:
    tt = 

200

In [5]:

ys_cpp, time_cpp, cpp_success = solve_with_cpp(f, t_span, y0)



Solving with C++ RK23
